# Microsoft Malware Detection XGBoost Blending CV predictions

![](https://winbuzzer.com/wp-content/uploads/2016/02/Microsoft-Logo-1-1.jpg)

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRbuYMzpgoRh2tLUj_EVV0z7gtIKwfJfZ7G-DP5dscAvSqcgSR_OQ)

![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dc/Cog-scripted-svg-green.svg/537px-Cog-scripted-svg-green.svg.png)


The Goal For this Kernel is to blend / mix predictions from different folds when training the XGBoost classifier. The idea stems from different Kernels which I want to credit for such as:

- https://www.kaggle.com/artgor/is-this-malware-eda-fe-and-lgb-updated



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns

%matplotlib inline

import time


# due to Kaggle memory limitations and the enormous dataset size, a sample from the whole
# trainset will be used for ML modeling
train_sample_fraction = 0.2


# another global variable that must be defined is the NA values rate / theshold to ommit columns with
# NA values that pass this rate
na_rate_threshold = 0.9

# theshold to remove columns with unbalanced features to their values 
unbalanced_feature_rate_threshold = 0.9

# Any results you write to the current directory are saved as output.

In [ ]:
# I am grateful for the help of author of this kernel for the main idea to load the dataset and save memory space!!
# https://www.kaggle.com/theoviel/load-the-totality-of-the-data

dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
%%time
train = pd.read_csv('../input/train.csv', dtype=dtypes)

In [ ]:
good_cols = list(train.columns)

for col in train.columns:
    
    # remove columns with high NA rate
    na_rate = train[col].isnull().sum() / train.shape[0]
    
    # remove columns with high Unbalanced values rate
    unbalanced_rate = train[col].value_counts(normalize=True, dropna=False).values[0]
    
    if na_rate > na_rate_threshold:
        good_cols.remove(col)
    elif unbalanced_rate > unbalanced_feature_rate_threshold:
        good_cols.remove(col)

In [ ]:
good_cols

In [ ]:
train = train[good_cols]

In [ ]:
import gc

gc.collect()

In [ ]:
categorical_columns = list(train.loc[:, train.dtypes =="category"].columns)
numerical_and_binary_columns = list(train.loc[:, train.dtypes !="category"].columns)
numerical_columns = numerical_and_binary_columns

categorical_columns.remove("MachineIdentifier")

binary_columns = []
for col in (numerical_and_binary_columns):
    if train[col].nunique() == 2:
        binary_columns.append(col)
        numerical_columns.remove(col)

## Machine Learning Modeling and Tuning

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRB40XiqXJooyGCgeV5-6DLkKEdDgFbvefkDgLMFG7_f4Sl0VQt)

![](http://www.pevco.com/wp-content/uploads/2015/05/gears-clear.png)

In [ ]:
train_sample = train.sample(frac=train_sample_fraction, random_state=42)
del train
gc.collect()

In [ ]:
train_sample.shape

In [ ]:
test_dtypes = {k: v for k, v in dtypes.items() if k in good_cols}

# get all columns except
test = pd.read_csv('../input/test.csv', dtype=test_dtypes, usecols=good_cols[:-1])

#test = reduce_mem_usage(test)

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
train_sample = train_sample.drop(['MachineIdentifier'], axis=1)
test = test.drop(['MachineIdentifier'], axis=1)

In [ ]:
train_sample = train_sample.reset_index(drop=True)

### Filling NA values with the statistical Mode

In [ ]:
modes = train_sample.mode()

for col in train_sample.columns:
    train_sample[col] = np.where(train_sample[col].isnull(), modes[col], train_sample[col])

del modes

In [ ]:
modes_test = test.mode()

for col in test.columns:
    test[col] = np.where(test[col].isnull(), modes_test[col], test[col])

#train_sample.shape
del modes_test

### Concatenate both train_sample and test sets before label encoding

In [ ]:
train_shape = train_sample.shape
test_shape = test.shape

train_and_test = pd.concat([train_sample,test], axis="rows", sort=False)

del train_sample
del test
gc.collect()

In [ ]:
train_and_test.head()

In [ ]:
train_and_test.tail()

### Encode the Categorical features before machine learning modeling

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def MultiLabelEncoder(columnlist,dataframe):
    for i in columnlist:
        #print(i)
        labelencoder_X=LabelEncoder()
        dataframe[i]=labelencoder_X.fit_transform(dataframe[i])

MultiLabelEncoder(categorical_columns, train_and_test)

In [ ]:
gc.collect()

### Back to train and test set after Label Encoding

In [ ]:
train_sample = train_and_test[0:train_shape[0]]
test = train_and_test[(train_shape[0]):(train_and_test.shape[0]+1)]

In [ ]:
del train_and_test

### Remove the HasDetections columns from test set, it has been added during dataframe concatenation.

In [ ]:
test = test.drop(["HasDetections"], axis = 1)

In [ ]:
y = train_sample['HasDetections']
X = train_sample.drop(['HasDetections'], axis=1)

In [ ]:
del train_sample
gc.collect()

### XGBoost Baseline model and blending Folds

In [ ]:
# main idea:
# https://www.kaggle.com/infinitewing/k-fold-cv-xgboost-example-0-28?scriptVersionId=1553202

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn import metrics
import time
import random

K = 5
index = 0
predictions_proba_test_list = np.zeros(len(test))
fold_auc_list = []
fold_accuracy_list = []

kf = KFold(n_splits = K, random_state = 42, shuffle = True)

for train_index, test_index in kf.split(X):
    
    print("Fold:", index)
    index = index + 1
    
    train_X, valid_X = X.iloc[train_index, :], X.iloc[test_index, :]
    train_y, valid_y = y[train_index], y[test_index]
    
    new_seed =  random.randint(1, 2000)
    
    clf_xgb = xgb.XGBClassifier(learning_rate=0.03, 
                            n_estimators=1300, 
                            max_depth=8,
                            min_child_weight=4,
                            gamma=0,
                            subsample=0.8,
                            colsample_bytree=0.7,
                            objective= 'binary:logistic',
                            nthread=-1,
                            scale_pos_weight=1,
                            reg_alpha = 0.1,
                            reg_lambda = 1,
                            seed=new_seed)

    clf_xgb.fit(train_X, train_y, eval_set=[(train_X, train_y), (valid_X, valid_y)], 
            early_stopping_rounds=100, eval_metric='auc', verbose=100)
    
    temp_predictions_proba_test_list = []

    # read test set in chunks
    chunck = 400000
    test_times = test.shape[0] // chunck
    test_rest = test.shape[0] % chunck

    for i in  np.arange(0,(chunck * (test_times+1)), chunck):
        
        # create predictions in chunks due ot memory limitations
        predictions_proba_test = list(clf_xgb.predict_proba(test[i:(i+chunck)])[:,1])
        temp_predictions_proba_test_list.append(predictions_proba_test)
        #print("times:", i)


    # flatten the list of lists
    temp_predictions_proba_test_list = [y for x in temp_predictions_proba_test_list for y in x]
    
    
    #print(np.shape(predictions_proba_test_list))
    predictions_proba_test_list = [sum(x) for x in zip(predictions_proba_test_list, temp_predictions_proba_test_list)]
    #print(test.shape)
    #print(np.shape(predictions_proba_test_list))

    
    predictions = clf_xgb.predict(valid_X, ntree_limit=clf_xgb.n_estimators)

    print()
    print(classification_report(valid_y, predictions))

    print()
    print("accuracy_score", accuracy_score(valid_y, predictions))
    
    predictions_probas = clf_xgb.predict_proba(valid_X)[:,1]
    print("auc score", roc_auc_score(valid_y, predictions_probas))
    print()
    
    fold_accuracy_list.append(accuracy_score(valid_y, predictions))
    fold_auc_list.append(roc_auc_score(valid_y, predictions_probas))

print()
print("Mean auc:", np.mean(fold_auc_list))
print("Std auc:", np.std(fold_auc_list))

print("Mean accuracy:", np.mean(fold_accuracy_list))
print("Std accuracy:", np.std(fold_accuracy_list))

gc.collect()

In [ ]:
predictions_proba_test_list = [x / kf.n_splits for x in predictions_proba_test_list]

In [ ]:
from sklearn.metrics import confusion_matrix
import scikitplot as skplt

sns.set(rc={'figure.figsize':(8,8)})
skplt.metrics.plot_confusion_matrix(valid_y, predictions, cmap="BrBG")

In [ ]:
sns.set(rc={'figure.figsize':(8,8)})
predictions_probas = clf_xgb.predict_proba(valid_X)
skplt.metrics.plot_roc(valid_y, predictions_probas)

In [ ]:
sns.set(rc={'figure.figsize':(8,8)})
skplt.metrics.plot_ks_statistic(valid_y, predictions_probas)

In [ ]:
sns.set(rc={'figure.figsize':(8,8)})
skplt.metrics.plot_precision_recall(valid_y, predictions_probas)

### Tuning
#### The following part is commented, I have run tuning in previous versions of this kernel and figured out the optimal (so far) values of the xgboost parameters.

### XGBoost Grid Search Part 1
#### Tuning parameters:
- 'max_depth'
- 'min_child_weight'

In [ ]:
# I may implement tuning in the future but I am afraid of variance - bias tradeoff
'''
#idea and a big thank you to https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
from sklearn.model_selection import GridSearchCV   #Perforing grid search

gc.collect()

param_test1 = {
    # based on previous personal kernels both parameters show better result having high numbers 
 'max_depth':[3, 5, 7, 9, 11],
 'min_child_weight':[1, 3, 5, 7, 9]
}
gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.05, n_estimators=70, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                                  objective= 'binary:logistic', nthread=-1, scale_pos_weight=1, reg_alpha = 0, 
                                                reg_lambda =1, seed=42), 
                        param_grid = param_test1, scoring='roc_auc', n_jobs=1, iid=False, cv=3, verbose = 1)

gsearch1.fit(xtrain, ytrain)
gsearch1.best_params_, gsearch1.best_score_
'''

### Prepare for submission

In [ ]:
del X
del y
del train_X
del train_y
del valid_X
del valid_y
del predictions
del predictions_probas
del temp_predictions_proba_test_list
del clf_xgb
gc.collect()

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv')
submission['HasDetections'] = predictions_proba_test_list
submission.to_csv('xgboost.csv', index=False)